In [1]:
from database import *
from pattern.en import conjugate, lemma, lexeme, PAST, SG, PRESENT

from Udep2Mono.binarization import BinaryDependencyTree
from Udep2Mono import polarization
from Udep2Mono.util import btreeToList
from copy import deepcopy

2020-12-21 22:07:44 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | gum       |
| pos       | gum       |
| lemma     | gum       |
| depparse  | gum       |
| ner       | ontonotes |

2020-12-21 22:07:44 INFO: Use device: gpu
2020-12-21 22:07:44 INFO: Loading: tokenize
2020-12-21 22:07:48 INFO: Loading: pos
2020-12-21 22:07:50 INFO: Loading: lemma
2020-12-21 22:07:50 INFO: Loading: depparse
2020-12-21 22:07:52 INFO: Loading: ner
2020-12-21 22:07:53 INFO: Done loading processors!


In [2]:
class ImplicativeGenerator:
    def __init__(self, length, kb, tree):
        self.kb = kb
        self.treeLog = []
        self.polarLog = []
        self.deptree = tree
        self.length = length

    def find_verbs(self, postags):
        verbs = []
        for word in postags:
            if 'VB' in postags[word][1]:
                verbs.append((word, postags[word][0]))
        return verbs
    
    def fix_tense(self, verb, pos):
        if pos == "VBD":
            return conjugate(verb=verb, tense=PAST, person=1)
        elif pos == "VBZ":
            return conjugate(verb=verb, tense=PRESENT, person=3)
        else:
            return verb

    def search(self):
        #verbs = self.find_verbs(postags)
        self.generate(self.deptree)

    
    def save_tree(self, tree=None):
        if tree is not None:
            generated, _, _, _ = btreeToList(tree, self.length, {}, 0)
        else:
            generated, _, _, _ = btreeToList(self.deptree, self.length, {}, 0)
        generated = '[%s]' % ', '.join(map(str, generated)).replace("'", "")
        generated = generated.replace(",", "")
        print("New tree: ", generated)

        if tree is not None:
            return deepcopy(self.deptree)
        else:
            return deepcopy(self.deptree)

    def generate(self, tree):
        if tree.val in ["ccomp", "xcomp"]:
            backup = deepcopy(tree)

            verb = conjugate(verb=tree.right.val, tense=PRESENT, person=1)
            pos = tree.right.npos

            impl_signs = self.kb.find({"Verb": verb})
            if impl_signs:
                sign = impl_signs[0]['Signature'].split('/')
                if sign[0] == "+" and sign[1] == "+":
                    self.treeLog.append(self.save_tree(tree.left.right))
                elif sign[0] == "+" and sign[1] == "-":
                    tree.val = tree.left.right.val
                    tree.mark = tree.left.right.mark
                    tree.id = tree.left.right.id
                    tree.right = tree.left.right.right
                    tree.right.val = self.fix_tense(tree.right.val, pos)
                    tree.left = tree.left.right.left
                    
                    self.treeLog.append(self.save_tree())

                    tree.val = backup.val
                    tree.mark = backup.mark
                    tree.id = backup.id
                    tree.left = deepcopy(backup.left)
                    tree.right = deepcopy(backup.right)
        else:
            if tree.left != "N":
                self.generate(tree.left)

            if tree.right != "N":
                self.generate(tree.right)       

In [3]:
#from Udep2Mono.dependency_parse import dependencyParse
#tree, postags, words = dependencyParse("All dogs eat food", parser="stanza")[0]

sentences = ["I managed to finish my homework", 
             "I recognized that this apple is good",
             "I realized that my homework is hard"]
annotations, _ = polarization.run_polarize_pipeline(
    sentences, verbose=2, parser="stanza")
print()
for annotation in annotations:
    annotated, original, polarized, postags, polarized_tree = annotation
    impgenerator = ImplicativeGenerator(len(original), db.implicative, polarized_tree)
    impgenerator.search()  

100%|██████████| 3/3 [00:01<00:00,  2.82it/s]
New tree:  [nsubj↑ [PRP i↑] [obj↑ [nmod:poss↑ [PRP$ my↑] [NN homework↑]] [VB finished↑]]]
New tree:  [nsubj↑ [det= [DT this=] [NN apple=]] [cop↑ [VBZ is↑] [JJ good↑]]]
New tree:  [nsubj↑ [nmod:poss= [PRP$ my↑] [NN homework↑]] [cop↑ [VBZ is↑] [JJ hard↑]]]

